lets pull the latest catalog of exoplanets from https://exoplanetarchive.ipac.caltech.edu/index.html

compare them to the various novae/SN lists we might come up with

check: are any of these exoplanets close to an ellipsoid edge? (maybe with a couple years?)

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time
from astropy import constants as const
import astropy.coordinates as coord
from astropy.table import Table

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size':18})
matplotlib.rcParams.update({'font.family':'serif'})

In [35]:
# Pfile = 'PS_2021.04.13_12.25.05.csv'
# planetary systems table:
# https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS&constraint=default_flag=1
Pfile = 'PS_2021.04.19_17.09.05.csv'

df = pd.read_csv(Pfile, comment='#')

df.shape

(29387, 100)

In [36]:
# TESS project candidates
# https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=TOI
Tfile = 'TOI_2021.04.19_17.09.56.csv'
dfT = pd.read_csv(Tfile, comment='#')

dfT.shape

(2645, 87)

In [55]:
# dfT['st_dist']
# okT = np.isfinite(dfT['st_dist'] )
# plt.scatter(dfT['ra'][okT], dfT['dec'][okT],s=10, alpha=0.5)
# sum(okT)

In [56]:
# dfT['tid'][okT].unique().size

In [54]:
# dfT.columns

# df['soltype'].unique()

In [57]:
# how many unique star systems can we explore?
# df['gaia_id'].unique().size

In [58]:
# how many unique star systems with good distances?
# df['gaia_id'][np.isfinite(df['sy_dist'])].unique().size

In [63]:
# use Pandas GroupBy & First to select only unique Gaia IDs (the first entries)
stars1 = df.groupby('gaia_id',as_index=False).first()

stars2 = dfT.groupby('tid',as_index=False).first()

print(sum(np.isfinite(stars1['sy_dist'])), 
      sum(np.isfinite(stars2['st_dist'])))

3041 2482


so this is actually the first real workflow shakedown! Real stars + real sources!

1. Prep Sources (Novae)
- read in list(s)
- put into same structures for matching

2. Prep the Stars:
- put ra,dec,dist into 3D SkyCoord object w/ units
- Time into astropy.Time object w/ units


3. Compute the Ellipse Positions
- full 3D distances, thank you astropy for doing this 
- then dsitances math from before... need to name this step better


In [73]:
# 1.
def make_objects(ra,dec,dist, Y, M, D):
    cc = SkyCoord(ra=ra, dec=dec, distance=dist)

    tt = Time({'year':Y, 'month':M, 'day':D}, 
              format='ymdhms')

    return cc,tt

def pull_sources(file='SOURCES_20210419.csv'):
    '''
    A simple wrapper to pull the "sources", aka the novae/SNe/original
    events as WE observed them, and put them into astropy structures.
    
    TO DO: sure wish we had errors, esp. on distance...
    '''
    src = pd.read_csv(file)

    cc, tt = make_objects(src['RA'].values*u.deg, 
                          src['Dec'].values*u.deg, 
                          src['dist'].values*u.pc, 
                          src['year'].astype('int').values, 
                          src['month'].astype('int').values, 
                          src['day'].astype('int').values)
    return cc, tt

cc_S, tt_S = pull_sources() # the event "sources"

In [74]:
# 2.
Sok1 = np.where(np.isfinite(stars1['sy_dist']))[0]
Sok2 = np.where(np.isfinite(stars2['st_dist']))[0]

# Coordinates for the stars/objects to monitor
cc_O = SkyCoord(ra=np.concatenate((stars1['ra'][Sok1].values, 
                                   stars2['ra'][Sok2].values)) * u.deg, 
                dec=np.concatenate((stars1['dec'][Sok1].values,
                                    stars2['dec'][Sok2].values)) * u.deg, 
                distance=np.concatenate((stars1['sy_dist'][Sok1].values,
                                         stars2['st_dist'][Sok2].values)) * u.pc)

# Times are... whatever we decide to observe over!
# * in the case of the ZTF "stream" they are the alert times, and match
#   1-to-1 with the coordinates
#
# * in the case of TESS data they might be the start/stop times of Sectors
#   that each source was oberved in (e.g. from the Web Viewing Tool)
#   https://heasarc.gsfc.nasa.gov/cgi-bin/tess/webtess/wtv.py
#
# * in the case of a proposal... let's just do 1 datapoint per year right now,
#   since that == 1 Lyr of distance within the SETI Ellipsoid

# tt_0 = Time( {'year':np.zeros(len(tt_0)) + 2021, 
#               'month':np.zeros(len(tt_0)) + 1, 
#               'day':np.zeros(len(tt_0)) + 1}, format='ymdhms')

In [79]:
j = 2019
tt_0 = Time( {'year':np.zeros(len(cc_O), dtype=np.int) + j, 
              'month':np.zeros(len(cc_O), dtype=np.int) + 1, 
              'day':np.zeros(len(cc_O), dtype=np.int) + 1}, format='ymdhms')
len(cc_O)


5523

In [102]:
# 3.

etol = 1. # ellipse tolerance (in lyr) (1 year for now... )
print('yr', '<0.07lyr', '<0.1lyr', '<1lyr')
# try a few years, just for an example
for j in (range(2019,2025)):
    tt_0 = Time( {'year':np.zeros(len(cc_O), dtype=np.int) + j, 
                  'month':np.zeros(len(cc_O), dtype=np.int) + 1, 
                  'day':np.zeros(len(cc_O), dtype=np.int) + 1}, format='ymdhms')

    # the smaller list is the novae/sources... 
    # loop over each Source, compute the ellipsoid distance to each star
    look1 = 0
    look2 = 0
    look3 = 0

    for k in range(tt_S.size):
        # time diff from Source[k] to all Stars/Objects
        # should be positive, for Novae in the past
        dt = tt_0 - tt_S[k]

        c = cc_S[k].distance.to('lyr') / 2 # the Source[k] foci distance
        a = (((dt.to('s') * const.c) / 2) + c).to('lyr') # the semi-major axis of Source[k] ellipse
        d2 = cc_O.separation_3d(cc_S[k]) # 3D dist between Source[k] and the Stars/Objects

        # the ellipsoid surface distance
        Edist = (cc_O.distance.to('lyr').value + d2.to('lyr').value) - (2 * a.to('lyr').value)

        # we should LOOK at sources if they are near the ellipsoid edges
        # (and if they have positive dt)
        look = (np.abs(Edist) < etol)  & (dt > 0)
        look1 += sum((np.abs(Edist) < 0.07)  & (dt > 0))
        look2 += sum((np.abs(Edist) < 0.1)  & (dt > 0))
        look3 += sum((np.abs(Edist) < 1.0)  & (dt > 0))

        # print the SN1987A results for each year
        if (tt_S[k].value[0] == 1987) & (tt_S[k].value[1] == 2):
            print('SN1987A, <0.07lyr: ', sum((np.abs(Edist) < 0.07)  & (dt > 0)))
            
    #     print(k, tt_S[k], sum(look))
    print(j, look1, look2, look3)

yr <0.07lyr <0.1lyr <1lyr
SN1987A, <0.07lyr:  2
2019 197 281 2932
SN1987A, <0.07lyr:  1
2020 210 295 2912
SN1987A, <0.07lyr:  0
2021 232 341 3056
SN1987A, <0.07lyr:  1
2022 198 286 2997
SN1987A, <0.07lyr:  2
2023 218 310 3007
SN1987A, <0.07lyr:  2
2024 197 281 3103
